## Steer Function Comparison
In this notebook, we examine the planners using different steer functions.

In [1]:
%load_ext autoreload
%autoreload 2
from mpb import MPB, MultipleMPB
from plot_stats import plot_planner_stats, plot_smoother_stats
from utils import latexify
from table import latex_table
from definitions import *
import matplotlib as mpl
import sys, os
mpl.rcParams['mathtext.fontset'] = 'cm'
# make sure to not use Level-3 fonts
mpl.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
from copy import deepcopy
from trajectory import visualize_grid

### Polygon Environments

In [2]:
time_limit_multiplier = 1
steer_function_times = [
    ('reeds_shepp', 15 * time_limit_multiplier),    
    ('dubins', 45 * time_limit_multiplier),    
    ('cc_reeds_shepp', 1.5 * 60 * time_limit_multiplier),    
    ('posq', 120 * time_limit_multiplier)
]

In [3]:
pools = {}
def benchmark_polygon(scenario: str, start: {str: float}, goal: {str: float}, robot_model: str = None):
    bench = "polygon_%s" % scenario
    
    pool = MultipleMPB()
    for steer_function, time_limit in steer_function_times:
        m = MPB()
        m["max_planning_time"] = time_limit
        m["env.start"] = start
        m["env.goal"] = goal
        m["env.type"] = "polygon"
        m["env.polygon.source"] = "polygon_mazes/%s.svg" % scenario[:-2]
        if robot_model:
            print("Using robot model %s." % robot_model)
            m["env.collision.robot_shape_source"] = robot_model
        m["benchmark.planning.sbpl_adstar"] = False
        m["benchmark.planning.sbpl_anastar"] = False
        m["benchmark.planning.sbpl_arastar"] = False
        m["benchmark.planning.sbpl_lazy_ara"] = False
        m["benchmark.planning.sbpl_mha"] = False
        m.set_steer_functions([steer_function])
        pool.benchmarks.append(m)
    pools[scenario] = pool
    run_result = pool.run_parallel(bench, processes=20, limit_memory=True)
    pool.merge(bench + "_total.json", make_separate_runs=True)
    return run_result

In [4]:
def generate_plots(scenario: str, steer_function: int):
    bench = "polygon_%s" % scenario
    steer_function_name = steer_function_times[steer_function][0]
    mpbs = [pool.benchmarks[steer_function] for pool_name, pool in pools.items() if pool_name.startswith(scenario)]
    total_name = "%s_total.json" % scenario
    MPB.merge(mpbs, total_name, make_separate_runs=True)
    bench_steer = "%s_%s" % (bench, steer_function_name)

    # plot planner stats
    try:        
        plot_planner_stats(total_name,
                           save_file="%s/%s_stats.pdf" % (bench, bench_steer),
                           max_plots_per_line=5,
                           metrics='path_length, curvature, planning_time, cusps, aggregate')
    except Exception as e:
        print("Error while plotting planner statistics:", e, file=sys.stderr)

Define 5 different start/goal configurations per polygon environment.

In [5]:
scenarios = [
    (
        "parking1", [
            ({"theta": 0, "x": 0, "y": -2.27}, {"theta": -1.58, "x": 7.72, "y": -7.72}),
            ({"theta": -1.58, "x": 7.72, "y": -7.72}, {"theta": 3.14, "x": 0, "y": -2.27}),
            ({"theta": -1.58, "x": 7.72, "y": -7.72}, {"theta": 1.58, "x": 10, "y": -7.72}),
            ({"theta": -1.58, "x": 7.72, "y": -7.72}, {"theta": 0, "x": 15, "y": -2.27}),
            ({"theta": 3.14, "x": 15, "y": -2.27}, {"theta": -1.58, "x": 7.72, "y": -7.72}),
        ]
    ), (
        "parking2", [
            ({"theta": 0, "x": 0, "y": -2.27}, {"theta": 1.57, "x": 10.91, "y": 2.73}),
            ({"theta": 1.57, "x": 10.91, "y": 2.73}, {"theta": 3.14, "x": 0, "y": -2.27}),
            ({"theta": 0, "x": 0, "y": -2.27}, {"theta": 1.57, "x": 13.5, "y": -7}),
            ({"theta": 1.57, "x": 13.5, "y": -7}, {"theta": 3.14, "x": 0, "y": -2.27}),
            ({"theta": 1.57, "x": 13.5, "y": -7}, {"theta": 1.57, "x": 10.91, "y": 2.73}),
        ]
    ), (
        "parking3", [
            ({"theta": 0, "x": 15.45, "y": -2.27}, {"theta": 0, "x": 3.4, "y": 0.34}),
            ({"theta": 0, "x": 5, "y": -2.27}, {"theta": 0, "x": 19.5, "y": -6}),
            ({"theta": 3.14, "x": 19.5, "y": -6}, {"theta": 3.14, "x": 5, "y": -2.27}),
            ({"theta": 3.14, "x": 19.5, "y": -2.5}, {"theta": 3.14, "x": 4.5, "y": 0.5}),
            ({"theta": 0, "x": 4.5, "y": 0.5}, {"theta": 0, "x": 19.5, "y": -2.5}),
        ]
    ), (
        "warehouse", [
            ({"theta": -1.58, "x": -2.5, "y": 4.55}, {"theta": -1.58, "x": 65, "y": 2.5}, "polygon_mazes/warehouse_robot.svg"),
            ({"theta": 0, "x": -2.5, "y": -55}, {"theta": 0, "x": 55, "y": 5.5}, "polygon_mazes/warehouse_robot.svg"),
            ({"theta": 1.58, "x": 14.5, "y": -28}, {"theta": 1.58, "x": 48, "y": -28}, "polygon_mazes/warehouse_robot.svg"),
            ({"theta": 3.14, "x": 65, "y": -55}, {"theta": 1.58, "x": 14.5, "y": 0}, "polygon_mazes/warehouse_robot.svg"),
            ({"theta": 3.14, "x": 3, "y": -5}, {"theta": 1.58, "x": 66, "y": -5}, "polygon_mazes/warehouse_robot.svg"),
        ]
    )
]

### Run benchmarks
For each steer function, run planners on 4 different polygon environments with 5 different start/goal configurations.

In [6]:
for i, (env, args) in enumerate(scenarios):
    for j, arg in enumerate(args):
        scenario = "%s_%i" % (env, j)
        benchmark_polygon(scenario, *arg)

Available memory: 12.43 GB, limiting each MPB process to 62.5% usage (7.77 GB).
Creating pool of 4 processes.
Created MPB from config polygon_parking1_0/polygon_parking1_0_1_config.json.
Created MPB from config polygon_parking1_0/polygon_parking1_0_0_config.json.
Created MPB from config polygon_parking1_0/polygon_parking1_0_2_config.json.
Created MPB from config polygon_parking1_0/polygon_parking1_0_3_config.json.
Running MPB with ID polygon_parking1_0_1 (log file at polygon_parking1_0/polygon_parking1_0_1.log)...
Running MPB with ID polygon_parking1_0_0 (log file at polygon_parking1_0/polygon_parking1_0_0.log)...
Running MPB with ID polygon_parking1_0_3 (log file at polygon_parking1_0/polygon_parking1_0_3.log)...
Running MPB with ID polygon_parking1_0_2 (log file at polygon_parking1_0/polygon_parking1_0_2.log)...


AttributeError: 'NoneType' object has no attribute 'items'

Plot statistics.

In [ ]:
print(pools.keys())
for steer in range(len(steer_function_times)):
    for env, _ in scenarios:
        generate_plots(env, steer)

Visualize trajectories.

In [ ]:
for env, args in scenarios:
    for j in range(1, len(args) + 1):
        suptitle = "Environment: %s   Scenario: %i" % (env, j)
        scenario = "%s_%i" % (env, j)
        visualize_grid("polygon_%s/polygon_%s_0_results.json" % (scenario, scenario),
                       suptitle=suptitle, save_file="%s.pdf" % scenario,
                       plot_every_nth_polygon=10)